### Glacier Ice and Snow Calculator.

Steps for the initial part of the project:-
1 - Gather Satellite images of a glacier mountainscape (Kenai Mountains) - Using Landsat images
2 - Present 3 different time periods (1999, 2010 - 2025 using landsat)
3 - Covert images into false colour composite
4 - Conduct a classification on both landsat scenes
5 - calculate area coverage (km2)
6 - present results


Potential expansions:-
- Expand the study period
- Lake changes

In [76]:
import os
import earthaccess
import geopandas as gpd
import rasterio as rio
import rasterio.merge
import shapely
import usgs

In [62]:
#First we are going to prepare the dataset by creating a Bounding Box and selecting the dates for the images we are going to use from earthaccess

KenaiMountains_BBox = (-161.5, 59.5, -158.0, 60.8)
years = ["1999-08-01", "2010-08-01", "2025-08-01"] #Summer months have been selected so that we can have clearer vision of glacier boundaries

#We will use the following satellites to acquire images over the Kenai Mountains for each respective time period
datasets = {
    "2000": "LANDSAT_7_C02_T1",  
    "2011": "LANDSAT_7_C02_T1",
    "2025": "LANDSAT_9_C2_L1" 
}

In [86]:
from usgs import api

# Open and read the token from the .usgs_token file
with open('C:/Users/couse/.usgs_user', 'r') as file:
    username = file.read()

with open('C:/Users/couse/.usgs_token', 'r') as file:
    password = file.read()
    
# Login to the API using the username and the token as password (or API key)
login_response = api.login(username, password, save=True)

# Now you are logged in, and you can proceed with other API calls
print("Successfully logged in to USGS API")
api_key = login_response['data']


Successfully logged in to USGS API


In [77]:
# List all available attributes and methods in the 'usgs.api' module
print(dir(usgs.api))

['FuturesSession', 'HTTPAdapter', 'Retry', 'TMPFILE', 'USGSAuthExpiredError', 'USGSError', 'USGS_API', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', '_check_for_usgs_error', '_create_session', '_get_api_key', 'dataset_download_options', 'dataset_filters', 'dataset_search', 'datetime', 'download_options', 'download_request', 'json', 'login', 'logout', 'os', 'payloads', 'requests', 'scene_metadata', 'scene_search']


In [89]:
# Search for all datasets available under a specific node (e.g., 'EE' = EarthExplorer)
response = api.dataset_search("landsat", "EE", api_key=api_key)

# Print available dataset names
if 'data' in response:
    print("Datasets you can access:")
    for dataset in response['data']:
        print(f"- {dataset['datasetAlias']}")
else:
    print("No datasets found or access denied.")

Datasets you can access:
- esat_etm_nopan
- esat_etm_pan
- ortho_mosaic_etm
- geos_5_fp_it
- geos_5_it
- lima
- landsat_mss_c2_l1
- landsat_tm_c2_l1
- landsat_tm_c2_l2
- landsat_ba_tile_c2
- landsat_dswe_tile_c2
- landsat_fsca_tile_c2
- landsat_fsca_tile_stat_c2
- landsat_ard_tile_c2
- landsat_etm_c2_l1
- landsat_etm_c2_l2
- landsat_ot_c2_l1
- landsat_ot_c2_l2
- landsat_band_files_c2_l1
- landsat_band_files_c2_l2
- lima_mosaic
- merra_2_c2
- ortho_mss_scene
- mss_film
- rbv_film
- sys_etm
- esat_tm
- tm_film
- ortho_mosaic
- viirs_atmos


In [ ]:
# Get filters for the dataset and node
filters = api.dataset_filters('landsat_ot_c2_l1', api_key)

# Show filter options
import json
print(json.dumps(filters, indent=2))

In [106]:
where = {
    "filter_id": '5e81f14f8d2a7c24',
    "value": "043",
    "operand": "="
}


In [111]:
# Perform the scene search with filters
results = api.scene_search(
    dataset="landsat_ot_c2_l1",  # Specify the dataset (e.g., LANDSAT_8_C1)
    start_date="2017-04-01",  # Start date for the scene search
    end_date="2017-05-01",    # End date for the scene search
    where=where,              # Apply the 'where' filter
    max_results=10,           # Limit the number of results
    api_key=api_key    # Provide your API key here
)
print(len(results))

6


In [10]:
help(api.scene_search)


Help on function scene_search in module usgs.api:

scene_search(dataset, max_results=5000, metadata_type=None, start_date=None, end_date=None, ll=None, ur=None, lat=None, lng=None, distance=100, where=None, starting_number=1, sort_order='DESC', api_key=None)
    :param dataset:
        USGS dataset (e.g. EO1_HYP_PUB, LANDSAT_8)
    :param lat:
        Latitude
    :param lng:
        Longitude
    :param distance:
        Distance in meters used to for a radial search
    :param ll:
        Dictionary of longitude/latitude coordinates for the lower left corner
        of a bounding box search. e.g. { "longitude": 0.0, "latitude": 0.0 }
    :param ur:
        Dictionary of longitude/latitude coordinates for the upper right corner
        of a bounding box search. e.g. { "longitude": 0.0, "latitude": 0.0 }
    :param start_date:
        Start date for when a scene has been acquired
    :param end_date:
        End date for when a scene has been acquired
    :where:
        Dictionary rep